# Resumo do código

### <u>Código que gera os ficheiros para estudo do comportamento da Delta</u>
---
O objectivo é receber dados da Delta e devolver um conjunto de métricas para prever roturas. Devolve um ficheiro que pode entrar no código 1 para juntar aos dados dos ninjas.

---
- Inputs

> __Dados completos da Delta em pastas de ficheiros__ (de azul a verde)
> - Stocks e trânsito, Sellout do dia anterior

                    ou

> __Ficheiro já completo__ (de vermelho a verde)
> - Stocks e trânsito, Sellout do dia anterior

- Outputs

> __Ficheiro com produtos em causa__ em formato Long

> __Métricas novas:__
> - Roturas de Stock e Pré-rotura
> - Sinal
> - Ciclos e Adequação de Stock
> - MSA (média de sellouts 10 dias antes)
> - STK (Stock disponível + trânsito)
> - (Novo) Balanço médio, mediano, liberal e conservador 
> - (Novo) Dias para a rotura de stock e de prateleira


In [58]:
%%time
import pandas as pd
import numpy as np
import datetime


def escrever_csv(dfa, nome): 
    dfa.to_csv('D:\\B&N Dados\\Delta\\Stocks\\StocksTotal\\%s.csv' %nome, index=False)

def escrever_txt(dfa, nome): 
    dfa.to_csv('D:\\B&N Dados\\Delta\\Stocks\\StocksTotal\\%s.txt' %nome, index=False, header=False)
    
def escrever_excel(dfa, nome):
    dfa.to_excel(f'D:\\B&N Dados\\Delta\\Stocks\\StocksTotal\\{nome}.xlsx' , index=False)

CPU times: total: 0 ns
Wall time: 0 ns


---

#  <span style="color:red"><u>Ler Ficheiro Completo</u> </span>

In [2]:
%%time

#Ler os ficheiros
df_2022 = pd.read_csv('D:\\B&N Dados\\Delta\\Stocks\\Stocks2022\\Stocks_Delta_2022_Limpo.csv')
df_2023 = pd.read_csv('D:\\B&N Dados\\Delta\\Stocks\\Stocks2023\\Stocks_Delta_2023_Limpo.csv')

# Juntar as bases
dataframes = [df_2022, df_2023]
df_Fusão = pd.concat(dataframes, ignore_index=True)
df_Fusão['DATA']= pd.to_datetime(df_Fusão['DATA'], format='%Y-%m-%d')

# Ficheiro de previsão
df_Prophet = pd.read_csv('D:\\B&N Dados\\Delta\\Forecast\\Prophet.csv')
df_Prophet['DATA']= pd.to_datetime(df_Prophet['DATA'], format='%Y-%m-%d')

df_XGBoost = pd.read_csv('D:\\B&N Dados\\Delta\\Forecast\\XGBoost.csv')
df_XGBoost['DATA']= pd.to_datetime(df_XGBoost['DATA'], format='%Y-%m-%d')

CPU times: total: 21.9 s
Wall time: 30.7 s


In [3]:
print(len(df_2023.DESC_ARTIGO.unique()))
print(len(df_2023.STORE.unique()))

181
334


# <font color=red>Opcional:</font> Definir produtos em causa

- Produtos específicos

In [59]:
%%time
# Ler ficheiro dos produtos e lojas para dataframe
df_produtos = pd.read_csv('D:\\B&N Dados\\Delta\\Piloto\\produtos.txt', header=None)
df_lojas = pd.read_csv('D:\\B&N Dados\\Delta\\Piloto\\lojas.txt', header=None)

# Passar para uma lista
produtos = df_produtos[0].tolist()
lojas = df_lojas[0].tolist()
produtos+=["CAFÉ DELTA Q MYTHIQ 10CAP", "CAFÉ DELTA Q MYTHIQ XL 40CAP", "CAFÉ DELTA Q QALIDUS 10CAP", "CAFÉ DELTA Q QALIDUS 40CAP",
                'CAFÉ DELTA Q QHARACTER 10CAP','CAFÉ DELTA Q QHARACTER 40CAP','CAFÉ DELTA Q DEQAFEINATUS 10CAP','CAFÉ DELTA Q DEQAFEINATUS XL 40CAP',
               'CAFÉ DELTA MOAGEM UNIVERSAL ANGOLA 220G','CAFÉ DELTA MOAGEM UNIVERSAL BRASIL 220G']
# Alterar o dataframe para apenas incluir os produtos e lojas em causa
dfFinal = df_Fusão[(df_Fusão["DESC_ARTIGO"].isin(produtos)) & (df_Fusão["STORE_NAME"].isin(lojas))].copy()

CPU times: total: 844 ms
Wall time: 857 ms


## <font color=red>Fim</font> 

In [60]:
dfFinal['SELLOUT'] = dfFinal.groupby(["STORE","EAN"])['SELLOUT_1_Dias_Antes'].shift(-1)
dfFinal['STOCK'] = dfFinal.groupby(["STORE","EAN"])['STOCK_1_Dias_Antes'].shift(-1)

- Fundir o Ficheiro da Delta com a previsão

In [61]:
%%time
# Prophet
dfFinal = pd.merge(dfFinal, df_Prophet[['DATA', 'STORE', 'DESC_ARTIGO', 'Prophet']], how="left", on=['DATA', 'STORE', 'DESC_ARTIGO',] )
# XGBoost
dfFinal = pd.merge(dfFinal, df_XGBoost[['DATA', 'STORE', 'DESC_ARTIGO', 'XGBoost']], how="left", on=['DATA', 'STORE', 'DESC_ARTIGO',] )

CPU times: total: 656 ms
Wall time: 616 ms


# <font color=green>Ficheiro Lido<font>

---

# Colunas de métricas interessantes

> - ROTURA

In [62]:
# Definir coluna de rotura (se stock menor ou igual a 0 e existe Linear)

dfFinal["ROTURA"] = np.where((dfFinal["STOCK"] <= 0) & (dfFinal["PRES_STOCK"] > 0), 1, 0)
dfFinal["ROTURA_1_Dias_Antes"] = np.where((dfFinal["STOCK_1_Dias_Antes"] <= 0) & (dfFinal["PRES_STOCK"].shift(1) > 0), 1, 0)


> - PRÉ_ROTURA

In [63]:
# Definir coluna de rotura (se stock menor ou igual a 0)

dfFinal["PRE_ROTURA"] = (dfFinal["STOCK"] < dfFinal["PRES_STOCK"]).astype(int)
dfFinal["PRE_ROTURA_1_Dias_Antes"] = (dfFinal["STOCK_1_Dias_Antes"] < dfFinal["PRES_STOCK"].shift(1)).astype(int)

# Colunas de métricas 30, 60, 120 e 180 dias antes

> - Volatilidade de Procura: <br>
coeficiente de variação

In [64]:
diasMetHist = [30, 60, 120, 180]

In [65]:
for i in diasMetHist:
    dfFinal[f"Percentagem_Volatilidade_{i}"] = (dfFinal.groupby(['DESC_ARTIGO', "STORE"])['SELLOUT_1_Dias_Antes'].transform(lambda x: x.rolling(window=i, min_periods=1).std())/
                             dfFinal.groupby(['DESC_ARTIGO', "STORE"])['SELLOUT_1_Dias_Antes'].transform(lambda x: x.rolling(window=i, min_periods=1).mean()))*100


In [ ]:
#Teste
colunas = dfFinal.filter(like='Percentagem_Volatilidade_').columns.tolist()

dfFinal[dfFinal.STORE==2][["DATA", "DESC_ARTIGO","STORE","SELLOUT_1_Dias_Antes"]+colunas[0:1]].head(31)

> - Percentagem de Rotura:

In [66]:
for i in diasMetHist:
    dfFinal[f"Percentagem_Roturas_{i}"] = (dfFinal.groupby(['DESC_ARTIGO', "STORE"])['ROTURA_1_Dias_Antes'].transform(lambda x: x.rolling(window=i, min_periods=1).mean()))*100

> - Percentagem de Supply:

In [67]:
# Sempre que é pedido abastecimento, fazer com que seja 1

dfFinal["New_Supply"] = np.where((dfFinal["EXPECTED"].shift(1)==0) & (dfFinal["EXPECTED"]>0), 1, 0)

In [68]:
for i in diasMetHist:
    dfFinal[f"Percentagem_Supply_{i}"] = (dfFinal.groupby(['DESC_ARTIGO', "STORE"])['New_Supply'].transform(lambda x: x.rolling(window=i, min_periods=1).mean()))*100


> - Efeito fim de semana

In [69]:
#dfFinal['SELLOUT_fds'] = dfFinal[dfFinal['DATA'].dt.weekday.isin([4,5,6])]["SELLOUT"].copy()
#dfFinal['SELLOUT_semana'] = dfFinal[dfFinal['DATA'].dt.weekday.isin([0,1,2,3])]["SELLOUT"].copy()
dfFinal['SELLOUT_fds_1_Dias_Antes'] = dfFinal[dfFinal['DATA'].dt.weekday.isin([5,6,0])]["SELLOUT_1_Dias_Antes"].copy()
dfFinal['SELLOUT_semana_1_Dias_Antes'] = dfFinal[dfFinal['DATA'].dt.weekday.isin([1,2,3,4])]["SELLOUT_1_Dias_Antes"].copy()

In [70]:
for i in diasMetHist:
    dfFinal[f"Percentagem_Efeito_Fds_{i}"] = ((dfFinal.groupby(['DESC_ARTIGO', "STORE"])['SELLOUT_fds_1_Dias_Antes']\
                                               .transform(lambda x: x.rolling(window=i, min_periods=1).mean())/
                                              (dfFinal.groupby(['DESC_ARTIGO', "STORE"])['SELLOUT_semana_1_Dias_Antes']\
                                               .transform(lambda x: x.rolling(window=i, min_periods=1).mean())))-1)*100


In [ ]:
#Teste
colunas = dfFinal.filter(like='Percentagem_Efeito_Fds_').columns.tolist()

dfFinal[dfFinal.STORE==2][["DATA", "DESC_ARTIGO","STORE", "SELLOUT_semana_1_Dias_Antes","SELLOUT_fds_1_Dias_Antes"]+colunas[0:1]].head(31)

> - Tempo médio inter-supply

In [71]:
dfFinal["InterSupply"] = np.where(dfFinal["EXPECTED"]==0, 1, 0)

groups = (dfFinal['InterSupply'] != dfFinal['InterSupply'].shift()).cumsum()
result = dfFinal.groupby(groups).agg({'DATA': 'first', 'DESC_ARTIGO': 'first', 'STORE': 'first', 'InterSupply': 'sum'}).reset_index(drop=True)
result = result[result['InterSupply'] > 0]

dfFinal = dfFinal.drop(columns=['InterSupply'])

dfFinal = pd.merge(dfFinal, result, how="left", on=["DATA","DESC_ARTIGO", "STORE"])

In [72]:
for i in diasMetHist:
    dfFinal[f"Percentagem_InterSupplyMed_{i}"] = (dfFinal.groupby(['DESC_ARTIGO', "STORE"])['InterSupply'].transform(lambda x: x.rolling(window=i, min_periods=1).mean()))*100


> - Tempo indisponível

In [73]:
'''O que vai acontecer é: no primeiro dia em que há rotura vai aparecer a soma de todos os dias com rotura a seguir a esse!
Todos os outros valores serão NaN para não serem considerados quando for feita a média. Assim a média corresponderá ao
número médio de dias em que se deixa um produto em rotura.'''

dfFinal["Dias_Indisponível_1_Dias_Antes"] = np.where(dfFinal["ROTURA_1_Dias_Antes"]==1, 1, 0)

In [74]:
for i in diasMetHist:
    dfFinal[f"Percentagem_Dias_Indisponível_{i}"] = (dfFinal.groupby(['DESC_ARTIGO', "STORE"])['Dias_Indisponível_1_Dias_Antes'].transform(lambda x: x.rolling(window=i, min_periods=1).mean()))*100


> - Percentagem de dias em Stock Borderline

In [75]:
dfFinal["Percentagem_Stock_Borderline_1_Dias_Antes"] = np.where(dfFinal["STOCK_1_Dias_Antes"]<0.2*dfFinal["PRES_STOCK"].shift(1), 1, 0)

In [76]:
for i in diasMetHist:
    dfFinal[f"Percentagem_Dias_Stock_Borderline_{i}"] = (dfFinal.groupby(['DESC_ARTIGO', "STORE"])['Percentagem_Stock_Borderline_1_Dias_Antes'].transform(lambda x: x.rolling(window=i, min_periods=1).mean()))*100


> - Percentagem de dias de Linear Incompleto

In [77]:
dfFinal["Percentagem_Linear_Incompleto_1_Dias_Antes"] = np.where(dfFinal["STOCK_1_Dias_Antes"]<dfFinal["PRES_STOCK"].shift(1), 1, 0)

In [78]:
for i in diasMetHist:
    dfFinal[f"Percentagem_Dias_Linear_Incompleto_{i}"] = (dfFinal.groupby(['DESC_ARTIGO', "STORE"])['Percentagem_Linear_Incompleto_1_Dias_Antes'].transform(lambda x: x.rolling(window=i, min_periods=1).mean()))*100


> - Percentagem de dias sem vendas

In [79]:
dfFinal["Sem_Vendas_1_Dias_Antes"] = np.where(dfFinal["SELLOUT_1_Dias_Antes"] == 0, 1, 0)

In [80]:
for i in diasMetHist:
    dfFinal[f"Percentagem_Dias_Sem_Vendas_{i}"] = (dfFinal.groupby(['DESC_ARTIGO', "STORE"])['Sem_Vendas_1_Dias_Antes'].transform(lambda x: x.rolling(window=i, min_periods=1).mean()))*100


> - Vendas perdidas

In [81]:
dfFinal['ROTURA_fds_1_Dias_Antes'] = dfFinal[dfFinal['DATA'].dt.weekday.isin([5,6,0])]["ROTURA_1_Dias_Antes"].copy()
dfFinal['ROTURA_semana_1_Dias_Antes'] = dfFinal[dfFinal['DATA'].dt.weekday.isin([1,2,3,4])]["ROTURA_1_Dias_Antes"].copy()

In [82]:
for i in diasMetHist:
    dfFinal[f"Vendas_Perdidas_em_{i}_Dias"] = ((dfFinal.groupby(['DESC_ARTIGO', "STORE"])['ROTURA_fds_1_Dias_Antes'].transform(lambda x: x.rolling(window=i, min_periods=1).sum())\
                                                  * dfFinal.groupby(['DESC_ARTIGO', "STORE"])['SELLOUT_fds_1_Dias_Antes'].transform(lambda x: x.rolling(window=i, min_periods=1).median())) \
                                                  + 
                                               (dfFinal.groupby(['DESC_ARTIGO', "STORE"])['ROTURA_semana_1_Dias_Antes'].transform(lambda x: x.rolling(window=i, min_periods=1).sum()) \
                                                  * dfFinal.groupby(['DESC_ARTIGO', "STORE"])['SELLOUT_semana_1_Dias_Antes'].transform(lambda x: x.rolling(window=i, min_periods=1).median())))
    

In [32]:
colunas = dfFinal.filter(like='Vendas_Perdidas_em_').columns.tolist()

dfFinal[dfFinal.STORE==2][["DATA", "DESC_ARTIGO","STORE","ROTURA_fds_1_Dias_Antes", "ROTURA_semana_1_Dias_Antes"]+colunas[0:1]].head(31)

,DATA,DESC_ARTIGO,STORE,ROTURA_fds_1_Dias_Antes,ROTURA_semana_1_Dias_Antes,Vendas_Perdidas_em_30_Dias
351,2022-01-01,BEBIDA CEREAIS DELTA C/20%CAFE FR 200G,2,0.0,NaN,NaN
352,2022-01-02,BEBIDA CEREAIS DELTA C/20%CAFE FR 200G,2,0.0,NaN,NaN
353,2022-01-03,BEBIDA CEREAIS DELTA C/20%CAFE FR 200G,2,0.0,NaN,NaN
354,2022-01-04,BEBIDA CEREAIS DELTA C/20%CAFE FR 200G,2,NaN,0.0,0.0
355,2022-01-05,BEBIDA CEREAIS DELTA C/20%CAFE FR 200G,2,NaN,0.0,0.0
356,2022-01-06,BEBIDA CEREAIS DELTA C/20%CAFE FR 200G,2,NaN,0.0,0.0
357,2022-01-07,BEBIDA CEREAIS DELTA C/20%CAFE FR 200G,2,NaN,0.0,0.0
358,2022-01-08,BEBIDA CEREAIS DELTA C/20%CAFE FR 200G,2,0.0,NaN,0.0
359,2022-01-09,BEBIDA CEREAIS DELTA C/20%CAFE FR 200G,2,0.0,NaN,0.0
360,2022-01-10,BEBIDA CEREAIS DELTA C/20%CAFE FR 200G,2,0.0,NaN,0.0


# Métricas 4, 5 e 10 antes:

- INSTRANSIT
- EXPECTED
- SELLOUT
- CICLOS
- Dias para Rotura
- Adequação

In [83]:
# Quantos dias antes:

diasMet = [1, 4, 5, 10]

> Função

In [84]:
# Função para colunas de dias anteriores
def dias(df, dia, coluna):         #dia é quantos dias antes
    a=int(dia)

    valores = df.groupby(['DESC_ARTIGO', 'STORE'])[coluna].transform(lambda x: x.shift(a))
    valores[:a] = np.nan
    
    df.loc[:,'%s_%s_Dias_Antes' % (coluna, a)] = valores

> - SELLOUTS

In [85]:
%%time
# Usar função para sellouts até 10 dias antes

for i in diasMet:
    dias(dfFinal, i, "SELLOUT")

CPU times: total: 281 ms
Wall time: 276 ms


> - STOCKS

In [86]:
# Usar função para Stocks até 10 dias antes

for i in diasMet:
    dias(dfFinal, i, "STOCK")

> > - Ordenar

> - INTRANSIT e EXPECTED

In [87]:
# Usar função para Trânsito até 10 dias antes


for i in diasMet:
    dias(dfFinal, i, "INTRANSIT")
    
for i in diasMet:
    dias(dfFinal, i, "EXPECTED")

> - STK

In [88]:
# STK do dia = soma dos stocks em loja com os stocks em trânsito no próprio dia

dfFinal["STK"] = dfFinal["STOCK"] + dfFinal["INTRANSIT"] + dfFinal["EXPECTED"]

for i in diasMet:
    dias(dfFinal, i, "STK")

> - MSA

In [89]:
# MSA do dia = média dos sellouts dos 10 dias anteriores ao dia em causa

dfFinal["MSA10"] = dfFinal.groupby(['DESC_ARTIGO', "STORE"])['SELLOUT'].shift(1).transform(lambda x: x.rolling(window=10).mean())
dfFinal["MSA10Dp"] = dfFinal.groupby(['DESC_ARTIGO', "STORE"])['SELLOUT'].shift(1).transform(lambda x: x.rolling(window=10).std())

for i in diasMet:
    dias(dfFinal, i, "MSA10")

    
dfFinal["MSA20"] = dfFinal.groupby(['DESC_ARTIGO', "STORE"])['SELLOUT'].shift(1).transform(lambda x: x.rolling(window=20).mean())
dfFinal["MSA20Dp"] = dfFinal.groupby(['DESC_ARTIGO', "STORE"])['SELLOUT'].shift(1).transform(lambda x: x.rolling(window=20).std())
  
for i in diasMet:
    dias(dfFinal, i, "MSA20")

> - CICLOS

In [90]:
# Coluna de Ciclos de reposição

dfFinal["CICLOS"] = dfFinal["STOCK"]/dfFinal["PRES_STOCK"]

for i in diasMet:
    dias(dfFinal, i, "CICLOS")

> - Dias para rotura de Stock

In [91]:
dfFinal = dfFinal.copy()
# Dias para a rotura mas com o Sellout médio (móvel) dos últimos 10 dias 
dfFinal["Dias_para_Rotura_Stock"] = dfFinal["STOCK"] / dfFinal["MSA10"]

for i in diasMet:
    dias(dfFinal, i, "Dias_para_Rotura_Stock")

> - Dias para rotura de Linear

In [92]:
# Definir a métrica: Preslinear / med(Sellouts 10 dias)
dfFinal['Dias_Duração_Linear'] = dfFinal["PRES_STOCK"] / dfFinal["MSA10"]

for i in diasMet:
    dias(dfFinal, i, "Dias_Duração_Linear")

> - Adequação de Stock

In [93]:
# Coluna de adequação de stock


dfFinal["Adequação"]= np.where(dfFinal["CICLOS"] > 1.1, "Stock Suficiente", 
                      np.where((dfFinal["CICLOS"] <= 1.1) & (dfFinal["INTRANSIT"]+dfFinal["EXPECTED"]+dfFinal["STOCK"]>=dfFinal["PRES_STOCK"]), "Stock Insuf c Forn Adequado", 
                      np.where((dfFinal["CICLOS"] <= 1.1) & (dfFinal["INTRANSIT"]+dfFinal["EXPECTED"]+dfFinal["STOCK"]<dfFinal["PRES_STOCK"]), "Stock Insuf c Forn Desadequado", 
                      "")))

for i in diasMet:
    dias(dfFinal, i, "Adequação")

>- Balance: sellout / soma stock disponível mais transito.

In [94]:
# Colunas de balanço


# Balance do dia = razão entre o sellout médio e o stock para o dia actual
dfFinal["Balance"] =  dfFinal["MSA10"] / dfFinal["STK"]

for i in diasMet:
    
    
    valores = i * dfFinal["Balance"].shift(i)
    valores[:i] = np.nan
    
    dfFinal.loc[:,'%s_%s_Dias_Antes' % ("Balance", i)] = valores
    
    


>- Balance optimizado

In [95]:


    
dfFinal["Balance_Optimized"] = np.where((dfFinal["MSA10Dp"] / dfFinal["MSA10"]) * 100 > 100, dfFinal["MSA20"] / dfFinal["STK"],
                                dfFinal["MSA10"] / dfFinal["STK"])   

for i in diasMet:
    
    
    valores = i * dfFinal["Balance_Optimized"].shift(i)
    valores[:i] = np.nan
    
    dfFinal.loc[:,'%s_%s_Dias_Antes' % ("Balance_Optimized", i)] = valores

> - Mediano é com o mínimo dos ultimos 10 dias

In [96]:
# Coluna de adequação de stock


# MSA_med do dia = mediana dos sellouts dos 10 dias anteriores (exclui o próprio dia)
# dfFinal["MdSA"] = dfFinal.groupby(['DESC_ARTIGO', "STORE"])['SELLOUT'].shift(1).transform(lambda x: x.rolling(window=10).median())


# Balance do dia = razão entre o sellout médio e o stock para o dia actual
dfFinal["Balance_Mediano"] =  dfFinal.groupby(['DESC_ARTIGO', "STORE"])['SELLOUT'].shift(1).transform(lambda x: x.rolling(window=10).median()) / dfFinal["STK"]

for i in diasMet:
    
    
    valores = i * dfFinal["Balance_Mediano"].shift(i)
    valores[:i] = np.nan
    
    dfFinal.loc[:,'%s_%s_Dias_Antes' % ("Balance_Mediano", i)] = valores



> - Liberal é com o mínimo dos ultimos 10 dias

In [97]:
# Liberal


# MSA do dia = média dos stocks dos 10 dias anteriores (exclui o próprio dia)
#dfFinal["LSA"] = dfFinal.groupby(['DESC_ARTIGO', "STORE"])['SELLOUT'].shift(1).transform(lambda x: x.rolling(window=10).min())


# Balance do dia = razão entre o sellout médio e o stock para o dia actual
dfFinal["Balance_Liberal"] =  dfFinal.groupby(['DESC_ARTIGO', "STORE"])['SELLOUT'].shift(1).transform(lambda x: x.rolling(window=10).min()) / dfFinal["STK"]

for i in diasMet:
    
    
    valores = i * dfFinal["Balance_Liberal"].shift(i)
    valores[:i] = np.nan
    
    dfFinal.loc[:,'%s_%s_Dias_Antes' % ("Balance_Liberal", i)] = valores



> - Conservador é com o máximo dos ultimos 10 dias

In [98]:
# Conservador

# MSA_max do dia = máximo dos stocks dos 10 dias anteriores (exclui o próprio dia)
#dfFinal["CSA"] = dfFinal.groupby(['DESC_ARTIGO', "STORE"])['SELLOUT'].shift(1).transform(lambda x: x.rolling(window=10).max())
 
    
# Balance do dia = razão entre o sellout médio e o stock para o dia actual
dfFinal["Balance_Conservador"] = dfFinal.groupby(['DESC_ARTIGO', "STORE"])['SELLOUT'].shift(1).transform(lambda x: x.rolling(window=10).max()) / dfFinal["STK"]

for i in diasMet:
    
    # Multiplicar os balances pelo número de dias+1 antes do dia actual
    valores = i * dfFinal["Balance_Conservador"].shift(i)
    valores[:i] = np.nan
    
    dfFinal.loc[:,'%s_%s_Dias_Antes' % ("Balance_Conservador", i)] = valores



> - Smart 5 dias antes

In [99]:
%%time
import pandas as pd
from datetime import datetime, timedelta


# Define a function to calculate weekday and weekend counts
def calculate_weekday_weekend_counts(date):
    weekday_count = 0
    weekend_count = 0
    
    for _ in range(5):
        if date.weekday() < 4:  # Monday to Thursday
            weekday_count += 1
        else:  # Friday to Sunday
            weekend_count += 1
        
        date += timedelta(days=1)
    
    return pd.Series({"CONTAGEM_SEMANA": weekday_count, "CONTAGEM_FIMSEMANA": weekend_count})

# Apply the function to each row in the DataFrame
dfFinal[["CONTAGEM_SEMANA", "CONTAGEM_FIMSEMANA"]] = dfFinal["DATA"].apply(calculate_weekday_weekend_counts)




CPU times: total: 25.8 s
Wall time: 25.8 s


In [100]:
# Smart

# 5 dias antes
dfFinal["Balance_Smart"] = ((
    (dfFinal.groupby(['DESC_ARTIGO', "STORE"])['SELLOUT_fds_1_Dias_Antes']\
    .transform(lambda x: x.rolling(window=30, min_periods=1).mean())* dfFinal['CONTAGEM_FIMSEMANA'])
    +
    (dfFinal.groupby(['DESC_ARTIGO', "STORE"])['SELLOUT_semana_1_Dias_Antes']\
    .transform(lambda x: x.rolling(window=30, min_periods=1).mean())* dfFinal['CONTAGEM_SEMANA']))
     / dfFinal["STK_1_Dias_Antes"])
dfFinal["Balance_Smart"] = np.where(dfFinal["STK"]<=0, -1, dfFinal["Balance_Smart"])
dfFinal["Balance_Smart_5_Dias_Antes"] = dfFinal["Balance_Smart"].shift(4)

In [101]:
dfFinal["RISCO"] = np.where((dfFinal.Balance_Smart > 2) & (dfFinal.CICLOS_5_Dias_Antes < 0.2) & (dfFinal.Percentagem_Roturas_120 > 5), 1,
                   np.where((dfFinal.Balance_Smart > 1) & (dfFinal.Balance_Smart < 2) & (dfFinal.CICLOS_5_Dias_Antes < 0.6) & (dfFinal.Percentagem_Roturas_120 < 3), 2, 3))

In [102]:
dfFinal["RISCO_ROTURA"] = np.where((dfFinal.Balance_Smart >= 12.5) | (dfFinal.Balance_Smart < 0), 1, 
                          np.where((dfFinal.Balance_Smart >= 1.1) & (dfFinal.Balance_Smart < 12.5), 2,
                          np.where((dfFinal.Balance_Smart >= 0.833) & (dfFinal.Balance_Smart < 1.1), 3,
                          np.where((dfFinal.Balance_Smart >= 0.087) & (dfFinal.Balance_Smart < 0.833), 4, 5))))         

In [103]:
dfVendedor=pd.read_excel("D:\\B&N Dados\\Delta\\Vendedor2.xlsx", sheet_name = "Lojas Sonae para o desafio")
dfVendedor = dfVendedor.rename(columns={"Cód. Loja":"STORE"})

# Criar coluna de reposição
dfFinal = pd.merge(dfFinal, dfVendedor[["STORE","Vendedor"]], how="left", on = "STORE")

# Escrever

- 0's e 1's

- Dias certos

In [104]:
#dfEscrever2022 = dfFinal.copy()
#dfEscrever2023 = dfFinal.loc[(dfFinal['DATA'] >= '2023-01-01') ].copy()

#df_DiasCertos = dfFinalLimitado.loc[(dfFinalLimitado['DATA'] >= '2023-01-01') ].copy()
#df_RoturasDiasCertos = dfFinalLimitadoRoturas.loc[(dfFinalLimitadoRoturas['DATA'] >= '2023-01-01') ].copy()

In [105]:
# Ficheiro Dia
dfEscreverDia = dfFinal[dfFinal.DATA == dfFinal.DATA.unique()[-1]].copy()

#Ficheiro Mês
dfEscreverMes = dfFinal[dfFinal.DATA >= dfFinal.DATA.unique()[-30]].copy()

- Passar para csv

In [106]:
escrever_excel(dfEscreverDia, "DataBaseDeltaDia")
escrever_excel(dfEscreverMes, "DataBaseDeltaMes")

In [86]:
dfEscrever2023[["DATA", "STORE","DESC_ARTIGO", "ROTURA", "CICLOS_5_Dias_Antes", "Balance_Smart_5_Dias_Antes", "Percentagem_Roturas_120"]]

,DATA,STORE,DESC_ARTIGO,ROTURA,CICLOS_5_Dias_Antes,Balance_Smart_5_Dias_Antes,Percentagem_Roturas_120
59,2022-03-01,1,BEBIDA CEREAIS DELTA C/20%CAFE FR 200G,0,1.325000,0.209459,0.0
60,2022-03-02,1,BEBIDA CEREAIS DELTA C/20%CAFE FR 200G,0,1.233333,0.203125,0.0
61,2022-03-03,1,BEBIDA CEREAIS DELTA C/20%CAFE FR 200G,0,1.066667,0.216814,0.0
62,2022-03-04,1,BEBIDA CEREAIS DELTA C/20%CAFE FR 200G,0,0.941667,0.198020,0.0
63,2022-03-05,1,BEBIDA CEREAIS DELTA C/20%CAFE FR 200G,0,0.841667,0.333333,0.0
...,...,...,...,...,...,...,...
87775,2023-05-06,459,CEVADA SOLÚVEL DELTA FRASCO 200G,0,8.375000,0.177384,0.0
87776,2023-05-07,459,CEVADA SOLÚVEL DELTA FRASCO 200G,0,7.046875,0.209246,0.0
87777,2023-05-08,459,CEVADA SOLÚVEL DELTA FRASCO 200G,0,4.921875,0.183099,0.0
87778,2023-05-09,459,CEVADA SOLÚVEL DELTA FRASCO 200G,0,3.453125,0.175337,0.0


In [114]:
%%time

#escrever_csv(dfEscrever2022, "FinalDataBaseDeltaTotal")
escrever_csv(dfEscrever2023[["DATA", "STORE","DESC_ARTIGO", "ROTURA", "CICLOS_5_Dias_Antes", "Balance_Smart_5_Dias_Antes", "Percentagem_Roturas_120"]], "FinalDataBaseDeltaTotal2023")

CPU times: total: 453 ms
Wall time: 445 ms


# Clickhouse

In [107]:
import clickhouse_connect
from clickhouse_driver import Client
from unidecode import unidecode

In [110]:
#Estabelecer a base a ser lida

baseCH = dfEscreverDia.copy()
#2023
tabela = "DataBaseDeltaDia"

In [111]:

client = clickhouse_connect.get_client(host='e28fluocjc.europe-west4.gcp.clickhouse.cloud', 
                                       port=8443, 
                                       username='default', 
                                       password='eKn4CWkTDFpi_',
                                       database='Delta')



###
baseCH['DATA']= pd.to_datetime(baseCH['DATA'], format='%Y-%m-%d')  # Passar para datetime
new_columns = [unidecode(col) for col in baseCH.columns]           # Tirar acentos e afins
baseCH.columns = new_columns                                       # Aplicar alterações da linha anterior
###

# Tipos de dados
data = ["DATA"]
texto = [col for col in baseCH.columns if baseCH[col].dtype == 'object']
inteiros = [col for col in baseCH.columns if baseCH[col].dtype == 'int64' or baseCH[col].dtype == 'int32']
floats = [col for col in baseCH.columns if baseCH[col].dtype == 'float64']


# Só floats é que permitem missing values
for col_name in inteiros:
    baseCH[col_name] = baseCH[col_name].astype(float)
# Missing values em strings estragam tudo
baseCH[texto] = baseCH[texto].fillna("-")


def schema(lista, tipo):
    result_list = [f"{element} {tipo}" for element in lista]
    return result_list

data1 = schema(data, "Date")
texto1 = schema(texto, "String")
inteiros1 = schema(inteiros, "Float64")
floats1 = schema(floats, "Float64")
total = tuple(data1 + texto1 + inteiros1 + floats1)

schema = ', '.join([column.replace("'", "") for column in total])

# Split the input string by commas
parts = schema.split(', ')
# Process each part and wrap the first word in double quotes
output_parts = []
for part in parts:
    words = part.split()
    if words:
        first_word = words[0]
        remaining_words = ' '.join(words[1:])
        output_part = f'"{first_word}" {remaining_words}'
        output_parts.append(output_part)
# Join the modified parts back into a string
schema = ', '.join(output_parts)



# Eliminar tabela no CH
client.command(f'DROP TABLE IF EXISTS {tabela}')

# Criar tabela no CH
client.command(f'''
    CREATE TABLE IF NOT EXISTS {tabela} (
        {schema}
        ) ENGINE = MergeTree
        ORDER BY (DATA)
''')

client.insert_df(tabela, baseCH, column_names=baseCH.columns.tolist())